In [ ]:
# imports
import torch
from stock_dataloader import create_stock_dataloader
from lstm import StockLSTM
from transformer import StockTransformer
from model_trainer import train_model, save_model, load_model
from evaluation import evaluate_model
from plots import plot_time_series

In [ ]:
# Create dataloader

# Hyperparameters
SEQ_LEN = 1250          # default 1250; window for set of time-series data points
BATCH_SIZE = 128        # default 128; uses 25-30% of 12GB NVIDIA GeForce RTX 4070 GPU
STOCKS_PER_BUCKET = 13  # default 13; number of stocks per category bucket
TRAIN_PER_BUCKET = 10   # default 10; number of training stocks per category bucket

stock_csv = 'selected_stocks_data.csv'
metadata_csv = 'selected_stocks_quality.csv'
stock_dataloader = create_stock_dataloader(stock_csv, metadata_csv, seq_len=SEQ_LEN, batch_size=BATCH_SIZE,
                                           stocks_per_bucket=STOCKS_PER_BUCKET, train_per_bucket=TRAIN_PER_BUCKET)
train_dataloader = stock_dataloader['train_loader']
train_dataset = stock_dataloader['train_dataset']
train_tickers = stock_dataloader['train_tickers']
eval_dataloader = stock_dataloader['eval_loader']
eval_dataset = stock_dataloader['eval_dataset']
eval_tickers = stock_dataloader['eval_tickers']
eval_scalers = stock_dataloader['eval_scalers']

In [ ]:
# Create LSTM Model

# Hyperparameters
INPUT_SIZE = 1          # default 1; based on data
HIDDEN_SIZE = 64        # default 64; analogous to D_MODEL; increase to 128 if underfitting
NUM_LAYERS = 2          # default 2, re-evaluate if underfitting
DROP_OUT = 0.2          # default 0.2; re-evaluate if overfitting

lstm_model = StockLSTM(input_size=INPUT_SIZE,
                  hidden_size=HIDDEN_SIZE,
                  num_layers=NUM_LAYERS,
                  dropout=DROP_OUT)

In [ ]:
# Create Transformer Model

# Hyperparameters
INP_DIM = 1             # default 1; based on data
D_MODEL = 64            # default 64; analogous to HIDDEN_SIZE; re-evaluate if underfitting
N_HEADS = 4             # default 4; 64/4 = 16 - standard ratio
N_LAYERS = 3            # default 3; re-evaluate if underfitting
DIM_FEEDFORWARD = 256   # default 256; 4x D_MODEL is standard
DROPOUT = 0.1           # default 0.1; re-evaluate if overfitting
OUTPUT_DIM = 1          # default 1; based on data - next-day closing price
MAX_LEN = 1500           # default 500; should be > SEQ_LEN

transformer_model = StockTransformer(inp_dim=INP_DIM,
                         d_model=D_MODEL,
                         n_heads=N_HEADS,
                         n_layers=N_LAYERS,
                         dim_feedforward=DIM_FEEDFORWARD,
                         dropout=DROPOUT,
                         output_dim=OUTPUT_DIM,
                         max_len=MAX_LEN)

In [ ]:
# Train model

# Hyperparameters
NUM_EPOCHS = 50         # default 50; increase if underfitting
LEARNING_RATE = 0.0001  # default 0.0001; 1e-3 exploded
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

model_choice = 'Transformer'   # Select 'LSTM' or 'Transformer'
model_save_name = f'Stock{model_choice}_ModelFull'
model_save = True
model_load = True

print(f"Training {model_choice} model on device: {DEVICE}")
if model_choice == 'LSTM':
    if model_load:
        try:
            trained_model = load_model(f'models\{model_save_name}')
        except FileNotFoundError as e:
            print(e)
            trained_model = train_model(lstm_model, train_dataloader, num_epochs=NUM_EPOCHS, learning_rate=LEARNING_RATE, device=DEVICE)
    else:
        trained_model = train_model(lstm_model, train_dataloader, num_epochs=NUM_EPOCHS, learning_rate=LEARNING_RATE, device=DEVICE)
    if model_save:
        save_model(trained_model, save_name=model_save_name)
elif model_choice == 'Transformer':
    if model_load:
        try:
            trained_model = load_model(f'models\{model_save_name}')
        except FileNotFoundError as e:
            print(e)
            trained_model = train_model(transformer_model, train_dataloader, num_epochs=NUM_EPOCHS, learning_rate=LEARNING_RATE, device=DEVICE)
    else:
        trained_model = train_model(transformer_model, train_dataloader, num_epochs=NUM_EPOCHS, learning_rate=LEARNING_RATE, device=DEVICE)
    if model_save:
        save_model(trained_model, save_name=model_save_name)
else:
    raise ValueError(f"{model_choice} is an invalid model choice. Please select 'LSTM' or 'Transformer'.")


In [ ]:
# Evaluate model
evaluate_model(trained_model, eval_dataloader, device=DEVICE)

In [ ]:
# Plot time-series

# From CSV
plot_time_series("selected_stocks_data.csv", tickers=['AMZN', 'EBAY', 'COKE'])

# Separate plots
plot_time_series("selected_stocks_data.csv", tickers=['AMZN', 'EBAY', 'COKE'], separate=True)

In [ ]:
# Need to update this plotting for model predictions vs actuals #
import numpy as np
trained_model.eval()
trained_model.to(DEVICE)

predictions = []
actuals = []

with torch.no_grad():
    for batch_x, batch_y in eval_dataloader:
        batch_x, batch_y = batch_x.to(DEVICE), batch_y.float().to(DEVICE)
        pred = trained_model(batch_x)  # Shape: [batch_size, 1]
        predictions.append(pred.cpu())
        actuals.append(batch_y.cpu())

# 2. Concatenate all predictions
all_predictions = torch.cat(predictions).squeeze().numpy()  # 1D array
all_actuals = torch.cat(actuals).squeeze().numpy()          # 1D array

# 3. Plot model vs actual
plot_time_series(all_actuals, tickers=["Actual"])
plot_time_series(all_predictions, tickers=["Predicted"])

# 4. Plot TOGETHER (1D tensor stacking)
combined = np.stack([all_actuals, all_predictions], axis=0)  # [2, N]
plot_time_series(combined, tickers=["Actual", "Predicted"])